In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os

# importing data from different runs
run_name = 'Run3'

tsg_screen_data = pd.read_excel('./TSGscreen/' + run_name+'.xlsx')
tsg_screen_data.dropna(axis=0)
tsg_screen_data.dtypes
tsg_screen_data['Time'] = pd.to_datetime(tsg_screen_data['Time'],format='%H:%M:%S')
tsg_screen_data.head()
feeder_data = pd.read_excel('./Feederdata/' + run_name+'.xlsx')
feeder_data = feeder_data.dropna(axis=1)
feeder_data['Time'] = pd.to_datetime(feeder_data['Time'],format='%H:%M:%S')
feeder_data.head()
eyecon_data = pd.read_excel('./Eyecondata/' + run_name+'.xlsx')
eyecon_data['Time'] = pd.to_datetime(eyecon_data['Time'],format='%H:%M:%S')
eyecon_data.head()
feed_eyecon = pd.merge_asof(eyecon_data,feeder_data,on='Time', tolerance=pd.Timedelta('2s'))
feed_eyecon = feed_eyecon.dropna(axis=0)
len(feed_eyecon)

combined_Data = pd.merge_asof(feed_eyecon,tsg_screen_data,on='Time', tolerance=pd.Timedelta('2s'))
combined_Data = combined_Data.dropna(axis=0)
combined_Data.columns.values
# plot_cols = [' D_v50', 'Torque','Zone 2','Zone 3','Zone 4','Zone 5','Zone 6','Zone 7','Zone 8']
# plot_cols = ['Zone 2','Zone 3','Zone 4','Zone 5','Zone 6','Zone 7','Zone 8']
plot_cols = [' D_v50', 'Torque']
time_arr = combined_Data.pop(combined_Data.columns.values[1])
plot_features = combined_Data[plot_cols]
plot_features.index = time_arr
_ = plot_features.plot(subplots=True)
combined_Data = combined_Data.drop([' TimeStamp','TimeStamp'],axis=1)
combined_Data = combined_Data.drop([' D_v10', ' D_v90', ' D_n10',' D_n50', ' D_n90', ' Median diameter',' Std deviation',' Shape mean', ' Shape RSD', '7 Massflow','7 Setpoint', '7 ScrewSpeed', '7 AveFeedFactor','7 DriveCommand'],axis=1)


In [ ]:
combined_Data_woTemp = combined_Data[['Mass flow rate','7 NetWeight','Liquid flow rate','Time index', 'Actual RPM','Torque',' D_v50']]
combined_Data_woTemp.head()

In [ ]:
column_indices = {name: i for i, name in enumerate(combined_Data.columns)}

n = len(combined_Data)
train_combined_Data = combined_Data_woTemp[0:int(n*0.6)]
test_combined_Data = combined_Data_woTemp[int(n*0.4):]

In [ ]:
scalers={}
train = train_combined_Data
for i in train_combined_Data.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train_combined_Data[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_combined_Data
for i in train_combined_Data.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

In [ ]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)


In [ ]:
n_past = 10
n_future = 5 
n_features = 7
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [ ]:
#
units = 12
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(units, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]


decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])


decoder_l1 = tf.keras.layers.LSTM(units, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
# model_e1d1.summary()

# reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=1000,validation_data=(X_test,y_test),batch_size=64,verbose=1)#,callbacks=[reduce_lr])

In [ ]:
plt.plot(np.array(history_e1d1.epoch),np.array(history_e1d1.history['loss']))
plt.plot(np.array(history_e1d1.epoch),np.array(history_e1d1.history['val_loss']))

In [ ]:
pred_e1d1=model_e1d1.predict(X_test)

In [ ]:
for index,i in enumerate(train_combined_Data.columns):
    scaler = scalers['scaler_'+i]
    pred_e1d1[:,:,index]=scaler.inverse_transform(pred_e1d1[:,:,index])
    pred_e1d1[:,:,index]=scaler.inverse_transform(pred_e1d1[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

In [ ]:
from sklearn.metrics import mean_absolute_error
for index,i in enumerate(train_combined_Data.columns):
  print(i)
  for j in range(1,5):
    print("Time ",j,":")
    print("MAE-E1D1 : ",mean_absolute_error(y_test[:,j-1,index],pred_e1d1[:,j-1,index]),end=", ")
  print()
  print()